In [ ]:
# ! pip install -e custom-diffusers/

In [1]:
from custom_diffusers import UNet2DConditionModel
from diffusers import DiffusionPipeline
from optimum.exporters.onnx import export
from optimum.onnxruntime import ORTStableDiffusionPipeline
from pathlib import Path
import onnx
import torch

/home/tran.duc.trungb/miniconda3/envs/triton-huggingface/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Download model

In [2]:
TORCH_DIR = Path("./exported-models/torch")

TORCH_DIR.mkdir(parents=True, exist_ok=True)


ONNX_DIR = Path("./exported-models/onnx")

ONNX_DIR.mkdir(parents=True, exist_ok=True)

In [3]:
MODEL_ID = "runwayml/stable-diffusion-v1-5"


In [4]:
# pipe = pipe.to("cuda")

if not (TORCH_DIR / "stable-diffusion-v1-5").exists():
    pipe = DiffusionPipeline.from_pretrained(
        MODEL_ID,
        torch_dtype=torch.float16,
        use_safetensors=True,
    )
    pipe.save_pretrained(TORCH_DIR / "stable-diffusion-v1-5")

# Export models to ONNX

In [5]:
SAVE_PATH = ONNX_DIR / "stable-diffusion-v1-5"

if not SAVE_PATH.exists():
    try:

        SAVE_PATH.mkdir(parents=True)
        pipeline = ORTStableDiffusionPipeline.from_pretrained(MODEL_ID, export=True)
        pipeline.save_pretrained(SAVE_PATH.as_posix())
    except Exception as e:
        print(e)
        SAVE_PATH.rmdir()

# else:
#     pipeline = ORTStableDiffusionPipeline.from_pretrained(
#         SAVE_PATH.as_posix(),
#     )

In [6]:
# prompt = "sailing ship in storm by Leonardo da Vinci"
# image = pipeline(prompt, num_inference_steps=2).images[0]

# pipeline.save_pretrained("./models/onnx/stable-diffusion-v1-5")

In [7]:

# pipe = DiffusionPipeline.from_pretrained(
#     "./torch-stable-diffusion-v1-5",
#     torch_dtype=torch.float16,
#     use_safetensors=True,
# )
# pipe = pipe.to("cuda")

# prompt = "a photo of an astronaut riding a horse on mars"
# image = pipe(prompt).images[0]



In [8]:
! rm ./exported-models/onnx/stable-diffusion-v1-5/unet/model.onnx_data
! rm ./exported-models/onnx/stable-diffusion-v1-5/unet/model.onnx

rm: cannot remove './exported-models/onnx/stable-diffusion-v1-5/unet/model.onnx_data': No such file or directory
rm: cannot remove './exported-models/onnx/stable-diffusion-v1-5/unet/model.onnx': No such file or directory


# Export Unet

In [9]:
unet = UNet2DConditionModel.from_pretrained("./exported-models/torch/stable-diffusion-v1-5/unet")

In [10]:
timesteps = torch.tensor([1, 2]).reshape(-1, 1).float()

session
prompt_embeds = torch.randn(2, 77, 768)

_ = unet(timestep=timesteps, sample=latent_model_input, encoder_hidden_states=prompt_embeds)

In [11]:
timesteps

tensor([[1.],
        [2.]])

In [12]:
from custom_diffusers.config import UNetOnnxConfig

In [13]:
onnx_config = UNetOnnxConfig(unet.config)

In [14]:
onnx_path = Path("./exported-models/onnx/stable-diffusion-v1-5/unet/model.onnx")

onnx_inputs, onnx_outputs = export(unet, onnx_config, onnx_path, onnx_config.DEFAULT_ONNX_OPSET)

Using framework PyTorch: 2.3.0+cu121
/home/tran.duc.trungb/triton/triton-playground/02.build_tensorrt_from_sd/custom-diffusers/src/custom_diffusers/unet_2d_condition.py:1228: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if dim % default_overall_up_factor != 0:


[('sample', torch.Size([2, 4, 64, 64])), ('timestep', torch.Size([2, 1])), ('encoder_hidden_states', torch.Size([2, 16, 768]))]


/home/tran.duc.trungb/miniconda3/envs/triton-huggingface/lib/python3.10/site-packages/diffusers/models/downsampling.py:137: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  assert hidden_states.shape[1] == self.channels
/home/tran.duc.trungb/miniconda3/envs/triton-huggingface/lib/python3.10/site-packages/diffusers/models/downsampling.py:146: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  assert hidden_states.shape[1] == self.channels
/home/tran.duc.trungb/miniconda3/envs/triton-huggingface/lib/python3.10/site-packages/diffusers/models/upsampling.py:149: Tr

In [15]:
onnx.checker.check_model(onnx_path.as_posix())

In [16]:
from optimum.exporters.onnx import validate_model_outputs

validate_model_outputs(
    onnx_config, unet, onnx_path, onnx_outputs, onnx_config.ATOL_FOR_VALIDATION
)


Validating ONNX model exported-models/onnx/stable-diffusion-v1-5/unet/model.onnx...


[('sample', torch.Size([2, 4, 64, 64])), ('timestep', torch.Size([2, 1])), ('encoder_hidden_states', torch.Size([2, 16, 768]))]


	-[✓] ONNX model output names match reference model (sample)
	- Validating ONNX Model output "sample":
		-[✓] (2, 4, 64, 64) matches (2, 4, 64, 64)
		-[✓] all values close (atol: 0.001)


In [3]:
import onnxruntime as ort
import numpy as np

In [6]:
session = ort.InferenceSession("./exported-models/onnx/stable-diffusion-v1-5/vae_decoder/model.onnx", providers=['CPUExecutionProvider'])

In [7]:
[
    (i.name, i.shape, i.type) for i in session.get_inputs()
]

[('latent_sample',
  ['batch_size', 'num_channels_latent', 'height_latent', 'width_latent'],
  'tensor(float)')]

In [9]:
session.run(
    None,
    {
        "latent_sample": np.random.randn(8, 4, 64, 64).astype(np.float32),
    }
)

[array([[[[ 3.85431945e-02,  8.64201039e-02,  9.67937335e-02, ...,
           -1.87966272e-01, -1.50687888e-01, -1.55924737e-01],
          [ 1.27990425e-01,  3.53805423e-02,  1.13523044e-01, ...,
           -1.79075912e-01, -1.28286734e-01, -1.77031934e-01],
          [ 7.17154741e-02,  1.01871401e-01,  7.33885840e-02, ...,
           -1.70409411e-01, -1.71882093e-01, -9.87272486e-02],
          ...,
          [ 9.36509445e-02,  1.26088843e-01,  3.58436722e-03, ...,
           -1.21429078e-01, -1.07200846e-01, -1.04507521e-01],
          [ 1.10540688e-01,  8.56413767e-02,  1.27397954e-01, ...,
           -1.43811285e-01, -1.33000195e-01, -6.96410239e-02],
          [ 1.92620531e-01,  2.74241455e-02,  6.12018891e-02, ...,
           -1.88904852e-01, -2.25699604e-01, -2.72514045e-01]],
 
         [[-3.01134009e-02,  1.47573836e-02,  4.02610153e-02, ...,
           -2.94286668e-01, -2.99780726e-01, -2.95068383e-01],
          [ 2.40223259e-02, -8.12322125e-02, -3.14687006e-03, ...,
     